# Experiments analysis

## Dependencies

In [ ]:
import json
import msgpack
import edge_sim_py
import pandas as pd
import networkx as nx
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

## Variables

In [ ]:
base_dir = "logs"

seed = "1"

last_step = 3600

variations = [
    "nodes=100;unique_images=08",
    "nodes=100;unique_images=32",
    "nodes=196;unique_images=16",
    "nodes=196;unique_images=64",
]

logs = [
    ("central", "central;{variation}"),
    ("community", "community12p;{variation}"),
    ("community", "community25p;{variation}"),
    ("p2p", "p2p;{variation}"),
    ("dynamic", "p2p;{variation}"),
    ("resource_aware_dynamic", "p2p;{variation}", "1"),
    ("resource_aware_dynamic", "p2p;{variation}", "2"),
    ("resource_aware_dynamic", "p2p;{variation}", "3"),
    ("resource_aware_dynamic", "p2p;{variation}", "4"),
]

logs_labels = ["Central", "Comm*", "Comm+", "P2P", "LMDyn", "MODyn (1)", "MODyn (2)", "MODyn (3)", "MODyn (4)"]

## Base functions

### User data: latency

In [ ]:
def get_user_data(variation, user_type=None):
    user_data = []

    for index, log in enumerate(logs):
        # Formatting log
        log = list(log)
        log[1] = log[1].format(variation=variation)

        simulation_data = {
            "Algorithm": logs_labels[index],
            "Mean Latency": 0,
        }

        # Opening file
        user_msgpack_file = (
            f"{base_dir}/algorithm={log[0]};dataset={log[1]};seed={seed}/User.msgpack"
            if len (log) == 2
            else f"{base_dir}/algorithm={log[0]};dataset={log[1]};seed={seed};replicas={log[2]}/User.msgpack"
        )
        user_file = open(user_msgpack_file, "rb")
        user_msgpack = msgpack.load(user_file)
        user_df = pd.DataFrame(user_msgpack)

        # Collecting information
        latency = (
            user_df[["Object", "Delays"]]
            if user_type is None
            else user_df[user_df["User Type"] == user_type][["Object", "Delays"]]
        )

        # Persisting information
        simulation_data["Mean Latency"] = latency["Delays"].mean()

        # Closing file
        user_file.close()

        # Appending data
        user_data.append(simulation_data)

    return pd.DataFrame(user_data)

In [ ]:
user_data_dfs = {
    variation: get_user_data(variation)
    for variation in variations
}

### Registry data: registry usage and number of provisioned registries

In [ ]:
def get_registry_data(variation):
    registry_data = []
    registry_data_per_time_step = {}

    for index, log in enumerate(logs):
        # Formatting log
        log = list(log)
        log[1] = log[1].format(variation=variation)

        simulation_data = {
            "Algorithm": logs_labels[index],
            "Values": [],
            "Values (w/o 0%)": [],
            "Mean": 0,
            "Mean Provisioned": 0,
        }

        # Opening file
        registry_msgpack_file = (
            f"{base_dir}/algorithm={log[0]};dataset={log[1]};seed={seed}/ContainerRegistry.msgpack"
            if len (log) == 2
            else f"{base_dir}/algorithm={log[0]};dataset={log[1]};seed={seed};replicas={log[2]}/ContainerRegistry.msgpack"
        )
        registry_file = open(registry_msgpack_file, "rb")
        registry_msgpack = msgpack.load(registry_file)
        registry_df = pd.DataFrame(registry_msgpack)

        # Collecting information
        registry_filtered_data = (
            registry_df[registry_df["P2P"] == True][["Object", "Provisioning", "Not Provisioning"]]
            if log[1] == "p2p"
            else registry_df[["Object", "Provisioning", "Not Provisioning"]]
        )
        registry_data_grouped = registry_filtered_data.groupby("Object").sum()
        registry_data_grouped["Total Steps"] = registry_data_grouped.sum(axis=1)
        registry_data_grouped["Provisioning Percentage"] = registry_data_grouped["Provisioning"] / registry_data_grouped["Total Steps"]
        registry_data_grouped["Not Provisioning Percentage"] = registry_data_grouped["Not Provisioning"] / registry_data_grouped["Total Steps"]
        registries_per_time_step = registry_df[registry_df["Time Step"] > 0].groupby(["Time Step"]).count()["Object"]

        # Persisting information
        simulation_data["Values"] = registry_data_grouped["Provisioning Percentage"]
        simulation_data["Values (w/o 0%)"] = registry_data_grouped[registry_data_grouped["Provisioning Percentage"] > 0]["Provisioning Percentage"]
        simulation_data["Mean"] = registry_data_grouped["Provisioning Percentage"].mean()
        simulation_data["Mean Provisioned"] = registry_data_grouped["Total Steps"].sum() / last_step
        registry_data_per_time_step[logs_labels[index]] = registries_per_time_step

        # Closing file
        registry_file.close()

        # Appending data
        registry_data.append(simulation_data)

    return pd.DataFrame(registry_data), pd.concat(registry_data_per_time_step, axis=1)

In [ ]:
registry_data_dfs = {
    variation: get_registry_data(variation)
    for variation in variations
}

### Edge Server data: computing resources utilization and disk utilization

In [ ]:
def normalize_cpu_and_memory(cpu, memory) -> float:
    """Normalizes the CPU and memory values.

    Args:
        cpu (float): CPU value.
        memory (float): Memory value.

    Returns:
        normalized_value (float): Normalized value.
    """
    normalized_value = (cpu * memory) ** (1 / 2)
    return normalized_value
    
def get_server_data(variation):
    computing_utilization_data = {}
    disk_utilization_data = {}
    server_data = []

    for index, log in enumerate(logs):
        # Formatting log
        log = list(log)
        log[1] = log[1].format(variation=variation)

        simulation_data = {
            "Algorithm": logs_labels[index],
            "Mean Disk Occupation per Server": 0,
        }
        
        # Opening file
        server_msgpack_file = (
            f"{base_dir}/algorithm={log[0]};dataset={log[1]};seed={seed}/EdgeServer.msgpack"
            if len (log) == 2
            else f"{base_dir}/algorithm={log[0]};dataset={log[1]};seed={seed};replicas={log[2]}/EdgeServer.msgpack"
        )
        server_file = open(server_msgpack_file, "rb")
        server_msgpack = msgpack.load(server_file)
        server_df = pd.DataFrame(server_msgpack)

        # Collecting information
        server_df = server_df[["Object", "CPU", "RAM", "CPU Demand", "RAM Demand", "Disk Demand", "Time Step"]]
        server_df["Normalized Utilization"] = server_df.apply(lambda row: normalize_cpu_and_memory(row["CPU Demand"], row["RAM Demand"])/normalize_cpu_and_memory(row["CPU"], row["RAM"]), axis=1)
        computing_utilization_per_time_step = server_df[server_df["Time Step"] > 0].groupby(["Time Step"])["Normalized Utilization"].mean()
        disk_utilization_per_time_step = server_df[server_df["Time Step"] > 0].groupby(["Time Step"])["Disk Demand"].sum()
        number_of_unique_servers = server_df["Object"].nunique()

        # Persisting information
        computing_utilization_data[logs_labels[index]] = computing_utilization_per_time_step
        disk_utilization_data[logs_labels[index]] = disk_utilization_per_time_step
        simulation_data["Total Disk Occupation"] = server_df["Disk Demand"].sum()
        simulation_data["Mean Disk Occupation per Server"] = simulation_data["Total Disk Occupation"] / (number_of_unique_servers * 3600)

        # Closing file
        server_file.close()

        # Appending data
        server_data.append(simulation_data)

    return pd.concat(computing_utilization_data, axis=1), pd.concat(disk_utilization_data, axis=1), pd.DataFrame(server_data)

In [ ]:
server_data_dfs = {
    variation: get_server_data(variation)
    for variation in variations
}

### Application data: provisioning time and reallocations

In [ ]:
def get_application_data(variation):
    service_data = []

    for index, log in enumerate(logs):
        # Formatting log
        log = list(log)
        log[1] = log[1].format(variation=variation)

        simulation_data = {
            "Algorithm": logs_labels[index],
            "Prov. Time Values": [],
            "Mean Prov. Time": 0,
            "Only Using Cache": 0,
            "Partially Using Cache": 0,
            "Not Using Cache": 0,
            "Total Migrations": 0,
            "Only Using Cache (%)": 0,
            "Partially Using Cache (%)": 0,
            "Not Using Cache (%)": 0,
        }

        # Opening file
        service_msgpack_file = (
            f"{base_dir}/algorithm={log[0]};dataset={log[1]};seed={seed}/Service.msgpack"
            if len (log) == 2
            else f"{base_dir}/algorithm={log[0]};dataset={log[1]};seed={seed};replicas={log[2]}/Service.msgpack"
        )
        service_file = open(service_msgpack_file, "rb")
        service_msgpack = msgpack.load(service_file)
        service_df = pd.DataFrame(service_msgpack)

        # Collecting information
        migrations_duration = service_df[service_df["Time Step"] == last_step]["Migrations Duration"].apply(pd.Series).stack().reset_index(drop=True)
        migrations_last_step = service_df[service_df["Time Step"] == last_step][["Object", "Migrations (Only Cache)", "Migrations (Partial Cache)", "Migrations (No Cache)"]]

        # Persisting information
        simulation_data["Prov. Time Values"] = service_df[service_df["Time Step"] == last_step]["Migrations Duration"].apply(pd.Series).stack().reset_index(drop=True)
        simulation_data["Mean Prov. Time"] = migrations_duration.mean()
        simulation_data["Only Using Cache"] = migrations_last_step["Migrations (Only Cache)"].sum()
        simulation_data["Partially Using Cache"] = migrations_last_step["Migrations (Partial Cache)"].sum()
        simulation_data["Not Using Cache"] = migrations_last_step["Migrations (No Cache)"].sum()
        simulation_data["Total Migrations"] = simulation_data["Only Using Cache"] + simulation_data["Partially Using Cache"] + simulation_data["Not Using Cache"]
        simulation_data["Only Using Cache (%)"] = simulation_data["Only Using Cache"] / simulation_data["Total Migrations"]
        simulation_data["Partially Using Cache (%)"] = simulation_data["Partially Using Cache"] / simulation_data["Total Migrations"]
        simulation_data["Not Using Cache (%)"] = simulation_data["Not Using Cache"] / simulation_data["Total Migrations"]

        # Closing file
        service_file.close()

        # Appending data
        service_data.append(simulation_data)

    return pd.DataFrame(service_data)

In [ ]:
application_data_dfs = {
    variation: get_application_data(variation)
    for variation in variations
}

### Topology data: image replication

In [ ]:
def get_image_replication_data(log, variation):
    # Formatting log
    log = list(log)
    log[1] = log[1].format(variation=variation)
    
    # Opening file
    topology_msgpack_file = (
        f"{base_dir}/algorithm={log[0]};dataset={log[1]};seed={seed}/Topology.msgpack"
        if len (log) == 2
        else f"{base_dir}/algorithm={log[0]};dataset={log[1]};seed={seed};replicas={log[2]}/Topology.msgpack"
    )
    topology_file = open(topology_msgpack_file, "rb")
    topology_msgpack = msgpack.load(topology_file, strict_map_key=False)
    topology_df = pd.DataFrame(topology_msgpack)

    # Collecting information
    image_replication_data = topology_df[["Object", "Replication Data", "Time Step"]]
    image_replication_data = pd.json_normalize(image_replication_data["Replication Data"]).fillna(0)
    image_replication_data = image_replication_data.divide(image_replication_data.sum(axis=1), axis=0)

    # Sort headers
    image_replication_data = image_replication_data.reindex(sorted(image_replication_data.columns), axis=1)

    # Closing file
    topology_file.close()

    return image_replication_data

### Utilitary

In [ ]:
def customize_chart(
    ax,
    title: str = "",
    xlabel: str = "",
    ylabel: str = "",
    xticklabels: list = [],
    yticklabels: list = [],
    legend: dict = {},
):
    """Customizes the chart.
    
    Args:
        ax (matplotlib.axes.Axes): Axes object.
        title (str): Chart title.
        xticklabels (list): List of xtick labels.
        xlabel (str): Label of the x axis.
        yticklabels (list): List of ytick labels.
        ylabel (str): Label of the y axis.
        legend_labels (list): List of legend labels.
    """
    if title != "":
        ax.set_title(title)

    if xlabel != "":
        ax.set_xlabel(xlabel)

    if ylabel != "":
        ax.set_ylabel(ylabel)

    if xticklabels != []:
        ax.set_xticklabels(xticklabels)
    
    if yticklabels != []:
        ax.set_yticklabels(yticklabels)

    if legend != {}:
        ax.legend(**legend)

## Experiments Description

### Container images size distribution

In [ ]:
# Loading container images metadata
container_images = json.load(open("datasets/inputs/templates/container_images.json", "r"))

# Ignoring registry image
container_images = container_images[1:]

# Collecting size of images
image_sizes = [sum([layer["size"] for layer in image["layers"]])/1000000 for image in container_images]

# Plot size distribution
plt.rcParams.update({'font.size': 18})
fig, ax = plt.subplots(figsize=(10, 5))
ax.hist(image_sizes, bins=10, edgecolor="black", color="white")

# Customizing chart
customize_chart(
    ax,
    xlabel="Size (MiB)",
    ylabel="Number of container images",
)

# Saving figure
plt.savefig(f"{base_dir}/image-size-distribution.pdf", bbox_inches='tight')

### Container layers sharing information

In [ ]:
container_layers = [layer for image in container_images for layer in image["layers"]]

layers_metadata = {}
for layer in container_layers:
    if layer["digest"] not in layers_metadata:
        layers_metadata[layer["digest"]] = {"size": layer["size"], "images_sharing": 1}
    else:
        layers_metadata[layer["digest"]]["images_sharing"] += 1

sharing_info = {}
for layer_metadata in layers_metadata.values():
    if layer_metadata["images_sharing"] not in sharing_info:
        sharing_info[layer_metadata["images_sharing"]] = { "total_size": layer_metadata["size"] }
    else:
        sharing_info[layer_metadata["images_sharing"]]["total_size"] += layer_metadata["size"]

total_size = sum([layer_metadata["size"] for layer_metadata in layers_metadata.values()])
for key in sharing_info:
    sharing_info[key]["percentage"] = round(sharing_info[key]["total_size"] / total_size * 100, 3)
    sharing_info[key]["total_size"] = round(sharing_info[key]["total_size"] / 1000000, 3)

layer_sharing_df = pd.DataFrame(sharing_info).T.sort_index()
layer_sharing_df.reset_index(inplace=True)
layer_sharing_df.columns = ["Number of images sharing", "Total size (MiB)", "Percentage of total size (%)"]
layer_sharing_df["Number of images sharing"] = layer_sharing_df["Number of images sharing"].astype(str)
layer_sharing_df = pd.concat([layer_sharing_df, pd.DataFrame({"Number of images sharing": "Total", "Total size (MiB)": layer_sharing_df["Total size (MiB)"].sum(), "Percentage of total size (%)": 100}, index=[0])])

layer_sharing_latex_string = layer_sharing_df.to_latex(
    float_format="%.3f",
    column_format="lcc",
    escape=True,
    caption="AWS Deep Learning layer-sharing information.",
    label="tab:layer-sharing",
    index=False
).replace(
    "\\begin{table}", "\\begin{table}\n\\centering"
).replace(
    "Total &", "\midrule\nTotal &"
)

with open("logs/layer_sharing.tex", "w") as file:
    file.write(layer_sharing_latex_string)

### Datasets

In [ ]:
def draw_datasets(target_variation: str):
    datasets = [
        "central",
        "community12p",
        "community25p",
        "p2p",
    ]

    dataset_labels = [
        "Central",
        "Community (12%)",
        "Community (25%)",
        "P2P",
    ]

    base_station_hatch = ""
    edge_server_hatch = "////"
    p2p_registry_hatch = "xxxx"
    full_registry_hatch = "**"

    plt.rcParams.update({'axes.titlesize': 18, 'legend.fontsize': 18})
    fig, ax = plt.subplots(2, 2, figsize=(15, 10))

    for index, dataset in enumerate(datasets):
        simulator = edge_sim_py.Simulator()
        simulator.initialize(input_file=f"datasets/{dataset};{target_variation}.json")
        topology = edge_sim_py.Topology.first()
        positions = {
            node: node.coordinates
            for node in topology.nodes()
        }

        # Gathering the coordinates of edge server
        edge_server_coordinates = [edge_server.coordinates for edge_server in edge_sim_py.EdgeServer.all()]

        # Gathering the coordinates of container registries
        full_registry_coordinates = [
            edge_server.coordinates
            for edge_server
            in edge_sim_py.EdgeServer.all()
            if len([registry for registry in edge_server.container_registries if registry.available and not registry.p2p_registry]) > 0
        ]
        p2p_registry_coordinates = [
            edge_server.coordinates
            for edge_server
            in edge_sim_py.EdgeServer.all()
            if len([registry for registry in edge_server.container_registries if registry.available and registry.p2p_registry]) > 0
        ]

        nx.draw_networkx_edges(topology, positions, ax=ax[index // 2][index % 2], edge_color="black", width=1)

        for node in topology.nodes():
            coordinates = node.coordinates
            hatch = full_registry_hatch if coordinates in full_registry_coordinates else p2p_registry_hatch if coordinates in p2p_registry_coordinates else edge_server_hatch if coordinates in edge_server_coordinates else base_station_hatch
            ax[index // 2][index % 2].scatter(
                coordinates[0],
                coordinates[1],
                s=250,
                c="white",
                marker="o",
                edgecolor="black",
                linewidth=0.5,
                hatch=hatch,
            )

        ax[index // 2][index % 2].set_title(dataset_labels[index])
        ax[index // 2][index % 2].axis("off")
        
    # add legend with custom hatches
    fig.legend(
        [
            mpl.patches.Patch(hatch=base_station_hatch, facecolor="white", edgecolor="black"),
            mpl.patches.Patch(hatch=edge_server_hatch, facecolor="white", edgecolor="black"),
            mpl.patches.Patch(hatch=p2p_registry_hatch, facecolor="white", edgecolor="black"),
            mpl.patches.Patch(hatch=full_registry_hatch, facecolor="white", edgecolor="black"),
        ],
        [
            "Only base station (BS)",
            "BS + edge server (ES)",
            "BS + ES + P2P registry",
            "BS + ES + fully replicated registry",

        ],
        loc="center",
        bbox_to_anchor=(0.5, 0.06),
        ncol=2,
    )

    plt.subplots_adjust(wspace=0.0, hspace=0.075)
    plt.savefig(f"{base_dir}/datasets-{target_variation}.pdf", bbox_inches='tight')

In [ ]:
draw_datasets("nodes=100;unique_images=08")

In [ ]:
draw_datasets("nodes=196;unique_images=64")

## Results

### Latency: comparative table

In [ ]:
latency_table = pd.DataFrame(
    {
        variation: user_data_dfs[variation]["Mean Latency"]
        for variation in variations
    }
)

latency_table.index = logs_labels

latency_table_latex_string = latency_table.to_latex(
    index=True,
    escape=True,
    header=False,
    label="tab:latency",
    column_format="lcccc",
    caption="Overall mean latency in time units (Equation~\\ref{eq:minimize-latency}).",
    float_format="{:0.2f}".format
).replace(
    "\\begin{table}",
    "\\begin{table}\n\centering"
).replace(
    "\\toprule",
    "\\toprule\nNodes & \multicolumn{2}{c}{100} & \multicolumn{2}{c}{196} \\\\\n\midrule\nUnique images & 08 & 32 & 16 & 64 \\\\"
)

with open(f"{base_dir}/table_latency.tex", "w") as f:
    f.write(latency_table_latex_string)

### Resource usage (provisioned registries): comparative table

In [ ]:
provisioned_registries_table = pd.DataFrame(
    {
        variation: registry_data_dfs[variation][0]["Mean Provisioned"]
        for variation in variations
    }
)

provisioned_registries_table.index = logs_labels

provisioned_registries_table_latex_string = provisioned_registries_table.to_latex(
    index=True,
    escape=True,
    header=False,
    label="tab:provisioned-registries",
    column_format="lcccc",
    caption="Mean number of provisioned registries per time step (Equation~\\ref{eq:minimize-computing}).",
    float_format="{:0.2f}".format
).replace(
    "\\begin{table}",
    "\\begin{table}\n\centering"
).replace(
    "\\toprule",
    "\\toprule\nNodes & \multicolumn{2}{c}{100} & \multicolumn{2}{c}{196} \\\\\n\midrule\nUnique images & 08 & 32 & 16 & 64 \\\\"
)

with open(f"{base_dir}/table_provisioned_registries.tex", "w") as f:
    f.write(provisioned_registries_table_latex_string)

### Resource usage (provisioned registries along the time): line chart

In [ ]:
plt.rcParams.update({'font.size': 14, 'axes.titlesize': 14, 'legend.fontsize': 12})
fig, ax = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)

for counter, (key, registry_data) in enumerate(registry_data_dfs.items()):
    registry_data[1].plot(ax=ax[counter // 2][counter % 2], legend=False, xticks=range(0, last_step + 1, 1200), yticks=range(0, 51, 10))
    customize_chart(
        ax[counter // 2][counter % 2],
        title=key,
    )
    ax[counter // 2][counter % 2].set_xlabel("")

handles, _ = plt.gca().get_legend_handles_labels()
fig.legend(handles, logs_labels, loc='center', ncols=5, bbox_to_anchor=(0.5, 0.94))
fig.text(0.07, 0.5, "Number of provisioned registries", va='center', rotation='vertical')
fig.text(0.5, 0.06, "Time step", ha='center')

plt.subplots_adjust(wspace=0.05, hspace=0.125)
plt.savefig(f"{base_dir}/number-of-provisioned-registries.pdf", bbox_inches='tight')

### Resource usage (server computing utilization along the time): line chart

In [ ]:
plt.rcParams.update({'font.size': 14, 'axes.titlesize': 14, 'legend.fontsize': 12})
fig, ax = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)

for counter, (key, server_data) in enumerate(server_data_dfs.items()):
    server_data[0].plot(ax=ax[counter // 2][counter % 2], legend=False, xticks=range(0, last_step + 1, 1200), yticks=[0.58, 0.6, 0.62, 0.64, 0.66, 0.68])
    customize_chart(
        ax[counter // 2][counter % 2],
        title=key,
    )
    ax[counter // 2][counter % 2].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, decimals=0))
    ax[counter // 2][counter % 2].set_xlabel("")

handles, _ = plt.gca().get_legend_handles_labels()
fig.legend(handles, logs_labels, loc='center', ncols=5, bbox_to_anchor=(0.5, 0.94))
fig.text(0.05, 0.5, "Normalized server utilization", va='center', rotation='vertical')
fig.text(0.5, 0.06, "Time step", ha='center')

plt.subplots_adjust(wspace=0.05, hspace=0.125)
plt.savefig(f"{base_dir}/normalized-mean-server-utilization-computing.pdf", bbox_inches='tight')

### Provisioning time: comparative table

In [ ]:
provisioning_time_table = pd.DataFrame(
    {
        variation: application_data_dfs[variation]["Mean Prov. Time"]
        for variation in variations
    }
)

provisioning_time_table.index = logs_labels

provisioning_time_table_latex_string = provisioning_time_table.to_latex(
    index=True,
    escape=True,
    header=False,
    label="tab:provisioning-time",
    column_format="lcccc",
    caption="Overall mean provisioning time in seconds (Equation~\\ref{eq:minimize-provisioning-time}).",
    float_format="{:0.2f}".format
).replace(
    "\\begin{table}",
    "\\begin{table}\n\centering"
).replace(
    "\\toprule",
    "\\toprule\nNodes & \multicolumn{2}{c}{100} & \multicolumn{2}{c}{196} \\\\\n\midrule\nUnique images & 08 & 32 & 16 & 64 \\\\"
)

with open(f"{base_dir}/table_provisioning_time.tex", "w") as f:
    f.write(provisioning_time_table_latex_string)

### Reallocations per type: stacker bar chart

In [ ]:
plt.rcParams.update({'font.size': 14, 'axes.titlesize': 14, 'legend.fontsize': 12})
fig, ax = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)

for counter, (key, application_data) in enumerate(application_data_dfs.items()):
    reallocations_df = application_data[["Algorithm", "Only Using Cache", "Partially Using Cache", "Not Using Cache"]]
    reallocations_df.plot.bar(x="Algorithm", stacked=True, ax=ax[counter // 2][counter % 2], legend=False)
    customize_chart(
        ax[counter // 2][counter % 2],
        title=key,
    )
    ax[counter // 2][counter % 2].set_xlabel("")
    ax[counter // 2][counter % 2].set_xticklabels(logs_labels, rotation=45, ha="right", rotation_mode="anchor")

handles, _ = plt.gca().get_legend_handles_labels()
labels = ["Only using cache", "Partially using cache", "Not using cache"]
fig.legend(handles, labels, loc='center', ncols=3, bbox_to_anchor=(0.5, 0.93))
fig.text(0.04, 0.5, "Number of reallocations", va='center', rotation='vertical')
fig.text(0.5, 0, "Algorithm", ha='center')

plt.subplots_adjust(wspace=0.05, hspace=0.125)
plt.savefig(f"{base_dir}/reallocations-per-type.pdf", bbox_inches='tight')

### Disk utilization: comparative table

In [ ]:
disk_utilization_table = pd.DataFrame(
    {
        variation: server_data_dfs[variation][2]["Mean Disk Occupation per Server"]
        for variation in variations
    }
)

disk_utilization_table.index = logs_labels

disk_utilization_table_latex_string = disk_utilization_table.to_latex(
    index=True,
    escape=True,
    header=False,
    label="tab:disk-utilization",
    column_format="lcccc",
    caption="Mean disk utilization per edge server and time step in MiB (Equation~\\ref{eq:minimize-storage}).",
    float_format="{:0.2f}".format
).replace(
    "\\begin{table}",
    "\\begin{table}\n\centering"
).replace(
    "\\toprule",
    "\\toprule\nNodes & \multicolumn{2}{c}{100} & \multicolumn{2}{c}{196} \\\\\n\midrule\nUnique images & 08 & 32 & 16 & 64 \\\\"
)

with open(f"{base_dir}/table_disk_utilization.tex", "w") as f:
    f.write(disk_utilization_table_latex_string)

### Disk utilization along the time: line chart

In [ ]:
def sizeof_fmt(x, pos):
    if x < 0:
        return ""
    for x_unit in ['MB', 'GB', 'TB']:
        if x < 1024.0:
            return "%3.1f %s" % (x, x_unit)
        x /= 1024.0

fig, ax = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)

for counter, (key, server_data) in enumerate(server_data_dfs.items()):
    server_data[1].plot(ax=ax[counter // 2][counter % 2], legend=False, xticks=range(0, last_step + 1, 1200))
    customize_chart(
        ax[counter // 2][counter % 2],
        title=key,
    )
    ax[counter // 2][counter % 2].yaxis.set_major_formatter(mticker.FuncFormatter(sizeof_fmt))
    ax[counter // 2][counter % 2].set_xlabel("")

handles, _ = plt.gca().get_legend_handles_labels()
fig.legend(handles, logs_labels, loc='center', ncols=5, bbox_to_anchor=(0.5, 0.94))
fig.text(0, 0.5, "Total disk utilization", va='center', rotation='vertical')
fig.text(0.5, 0.06, "Time step", ha='center')

plt.subplots_adjust(wspace=0.05, hspace=0.125)
plt.savefig(f"{base_dir}/total-disk-utilization.pdf", bbox_inches='tight')

### Registry usage (percentage of active steps): boxplot chart

In [ ]:
# TODO: check how to share labels

fig, ax = plt.subplots(2, 2, figsize=(10, 10), sharex=True, sharey=True)

for counter, (key, registry_data) in enumerate(registry_data_dfs.items()):
    registry_usage_df = registry_data[0][["Algorithm", "Values"]]
    ax[counter // 2][counter % 2].boxplot(registry_usage_df["Values"], labels=logs_labels)
    customize_chart(
        ax[counter // 2][counter % 2],
        title=key,
    )
    ax[counter // 2][counter % 2].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, decimals=0))

fig.text(0.05, 0.5, "Percentage of time active", va='center', rotation='vertical')
fig.text(0.5, 0, "Algorithm", ha='center')

for counter in range(2):
    ax[1][counter].set_xticklabels(ax[1][counter].get_xticklabels(), rotation=45, fontstyle="normal", ha="right", rotation_mode="anchor")

plt.subplots_adjust(wspace=0.05, hspace=0.125)
plt.savefig(f"{base_dir}/registries-percentage-of-time-active.pdf", bbox_inches='tight')

### Image replication

In [ ]:
target_variation = "nodes=100;unique_images=32"

image_replication_data = [
    get_image_replication_data(log, target_variation) for log in logs[3:]
]

headers = [
    image_replication_data[index].columns.values.tolist() for index in range(len(image_replication_data))
]

max_length = max([len(header) for header in headers])
header_with_max_length = headers[[len(header) for header in headers].index(max_length)]

global_colormap = mpl.colormaps["tab20c"].resampled(max_length)

In [ ]:
# TODO: fix when it does not start with zero
headers

In [ ]:
# plot stacked area chart for each algorithm in multiple figures using a single legend
plt.rcParams.update({'font.size': 14, 'axes.titlesize': 14, 'legend.fontsize': 12})
fig, ax = plt.subplots(2, 3, figsize=(20, 10), sharex=True, sharey=True)

for counter, image_replication_df in enumerate(image_replication_data):
    local_color_map = mpl.colors.ListedColormap(global_colormap.colors[:len(image_replication_df.columns)])
    image_replication_df.plot.area(ax=ax[counter // 3][counter % 3], legend=False, alpha=0.75, cmap=local_color_map, xticks=range(0, last_step + 1, 1200))
    ax[counter // 3][counter % 3].set_ylim([0, 1])
    ax[counter // 3][counter % 3].invert_yaxis()
    customize_chart(
        ax[counter // 3][counter % 3],
        title=logs_labels[counter+3],
    )

# percentage formatter
ax[0][0].yaxis.set_major_formatter(mticker.PercentFormatter(xmax=1, decimals=0))

# create a single colorbar for all figures
cax = fig.add_axes([0.91, 0.125, 0.02, 0.75])
fig.colorbar(mpl.cm.ScalarMappable(cmap=global_colormap), cax=cax, orientation="vertical", aspect=50, values=header_with_max_length, pad=0.1)
fig.text(0.96, 0.5, "Number of container image replicas", va='center', rotation='vertical')

fig.text(0.08, 0.5, "Percentage of images with number of replicas", va='center', rotation='vertical')
fig.text(0.5, 0.06, "Time step", ha='center')

plt.subplots_adjust(wspace=0.05, hspace=0.125)
plt.savefig(f"{base_dir}/image-replication-{target_variation}.pdf", bbox_inches='tight')